In [1]:
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
import pickle
import random

In [2]:
client_id = "f68226fa347346169cb5cdec05f72a8b"
client_secret ="e082fbec5984468f946e9e056e9083ce"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
song_clusters_df = pd.read_csv("../Data/songs_with_clusters.csv")

In [4]:
song_clusters_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,title,artist,hot_100,cluster
0,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.646,...,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4,flowers,miley cyrus,1,2
1,0.347,0.378,8,-9.960,1,0.0354,0.85700,0.000000,0.1180,0.610,...,5ReIs3aH4wfBjp93QOtWAl,spotify:track:5ReIs3aH4wfBjp93QOtWAl,https://api.spotify.com/v1/tracks/5ReIs3aH4wfB...,https://api.spotify.com/v1/audio-analysis/5ReI...,168609,4,kill bill,sza,1,1
2,0.517,0.675,6,-5.382,1,0.0357,0.45900,0.000000,0.1510,0.518,...,59uQI0PADDKeE6UZDTJEe8,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4,last night,morgan wallen,1,0
3,0.714,0.472,2,-7.375,1,0.0864,0.01300,0.000005,0.2660,0.238,...,3nqQXoyQOWXiESFLlDF1hG,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943,4,unholy,sam smith & kim petras,1,7
4,0.728,0.293,1,-13.657,0,0.0562,0.47900,0.904000,0.1180,0.152,...,4CpujwKsq4DxOIMiKr12m8,spotify:track:4CpujwKsq4DxOIMiKr12m8,https://api.spotify.com/v1/tracks/4CpujwKsq4Dx...,https://api.spotify.com/v1/audio-analysis/4Cpu...,217026,4,creepin',"metro boomin, the weeknd & 21 savage",1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,0.584,0.932,1,-3.501,1,0.3330,0.00106,0.002690,0.1290,0.204,...,3mT8SuFmvheAToweq9oLGt,spotify:track:3mT8SuFmvheAToweq9oLGt,https://api.spotify.com/v1/tracks/3mT8SuFmvheA...,https://api.spotify.com/v1/audio-analysis/3mT8...,274404,4,like a bitch - kill the noise remix,kill the noise,0,6
2485,0.576,0.451,8,-9.571,1,0.0444,0.46900,0.038900,0.0876,0.803,...,3yPD6CHGE1xdJBWyu6ZBKk,spotify:track:3yPD6CHGE1xdJBWyu6ZBKk,https://api.spotify.com/v1/tracks/3yPD6CHGE1xd...,https://api.spotify.com/v1/audio-analysis/3yPD...,172825,4,candy,dillon francis,0,1
2486,0.637,0.935,0,-2.467,1,0.1070,0.00857,0.003990,0.2140,0.468,...,6g10yOV7NssHq2XHpi6UgY,spotify:track:6g10yOV7NssHq2XHpi6UgY,https://api.spotify.com/v1/tracks/6g10yOV7NssH...,https://api.spotify.com/v1/audio-analysis/6g10...,207200,4,habit - dack janiels & wenzday remix,rain man,0,0
2487,0.557,0.992,1,-2.735,1,0.1330,0.00164,0.677000,0.0913,0.598,...,1zsBoEW57MoMPelCWSggHw,spotify:track:1zsBoEW57MoMPelCWSggHw,https://api.spotify.com/v1/tracks/1zsBoEW57MoM...,https://api.spotify.com/v1/audio-analysis/1zsB...,185600,4,first contact,twin moons,0,0


In [5]:
song_clusters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2489 entries, 0 to 2488
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      2489 non-null   float64
 1   energy            2489 non-null   float64
 2   key               2489 non-null   int64  
 3   loudness          2489 non-null   float64
 4   mode              2489 non-null   int64  
 5   speechiness       2489 non-null   float64
 6   acousticness      2489 non-null   float64
 7   instrumentalness  2489 non-null   float64
 8   liveness          2489 non-null   float64
 9   valence           2489 non-null   float64
 10  tempo             2489 non-null   float64
 11  type              2489 non-null   object 
 12  id                2489 non-null   object 
 13  uri               2489 non-null   object 
 14  track_href        2489 non-null   object 
 15  analysis_url      2489 non-null   object 
 16  duration_ms       2489 non-null   int64  


In [6]:
song_clusters_df = song_clusters_df.drop_duplicates()

In [7]:
def search_song(title, artist, limit=1):
    try:
        result = sp.search(q=title + artist, limit=limit)
        return result["tracks"]["items"][0]["id"]
    except:
        print('none')

In [8]:
def read_song_and_artist():
    title_s = input("Please enter the song title: ").lower()
    artist_s = input("Please enter the artist: "). lower 
    
    return artist_s, title_s

In [9]:
#title_s = input("Please Enter the song title: ").lower()
#artist_s = input("Please Enter the artist title: ").lower()

In [10]:
imput_s = search_song(title_s,artist_s,1)

NameError: name 'title_s' is not defined

In [ ]:
my_dict = sp.audio_features(imput_s)
my_dict

In [ ]:
imput_df = pd.DataFrame(my_dict)

In [ ]:
imput_s = imput_df.drop(['type','id','uri','track_href','analysis_url',
                                    'time_signature','key','mode'],axis=1)
imput_s

In [ ]:
def load(filename = "/Users/Renan/Documents/Ironhack/Week_6/Day_4/Morning/scalers/scaler.pickle"): 
    try: 
        with open(filename, "rb") as file: 
            return pickle.load(file) 
    except FileNotFoundError: 
        print("File ", filename, " not found!")

In [ ]:
scaler = load("../Scaler/scaler.pickle")
scaler

In [ ]:
best_model = load("../inertia/inertia8.pickle")

In [ ]:
song_scaled = scaler.transform(imput_s)

In [ ]:
song_c = best_model.predict(song_scaled)

In [ ]:
song_c

In [ ]:
cluster_n = song_clusters_df[song_clusters_df['cluster'] == song_c[0]] 
recommend = (cluster_n.iloc[:,18:20]).sample(5)

In [ ]:
recommend

In [ ]:
done = False 

scaler = load("../Scaler/scaler.pickle")
model = load("../inertia/inertia8.pickle")
songs = song_clusters_df

while not done: 
    
    title,artist = read_song_and_artist()
    
    id = search_song(artist, title)
    if not id:
        print("Unable to get song id")
        continue 
    
    af = get_audio_features(id)
    if not af:
        print("Unable to get audio features")
        continue
        
    af_scaled_df = pd.DataFrame(af)
    af_scaled_df.drop(['type','id','uri','track_href','analysis_url',
                                    'time_signature','key','mode'],axis=1, inplace=True)
    af_scaled = scaler.transform(af_scaled_df)
    af_scaled_df = pd.DataFrame(af_scaled, columns=af_scaled_df.columns)
    
    cluster = model.predict(af_scaled)[0]
    
    if not songs[(songs[id] == id) & (songs['hot_100'] == 1)].empty:
        recommended = songs[(songs['hot_100'] == 1) & (songs["cluster"] == cluster)].sample()
    else: 
        recommended = songs[(songs["hot_100"] == 0) & (songs["cluster"] == cluster)].sample()
        
    artist = recommended["artist"].values[0]
    title = recommended["title"].values[0]

        
    print("Recommended: {} - {}".format(artist, title))
    
    quit = input("Wanna quit? (y|n) ")
    if quit == "y": 
        done = True

In [ ]:
songs.artist.values[0]